In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext

sc = SparkContext("local", "FantaStats")
spark = SQLContext(sc)

In [36]:
from pyspark.sql.functions import *

def loadCSV(sdf, filename):
    df = sdf.load(filename)
    return df.withColumn("G2", regexp_replace("G2", " ", "-"))
    
def prefix(sdf, prefix):
      for c in sdf.columns:
          sdf = sdf.withColumnRenamed(c, '{}{}'.format(prefix, c))
      return sdf

In [44]:
csvContext = spark.read.format("csv").option("header", "true").option("delimiter", ";")

players_2015 = prefix(loadCSV(csvContext, "2015-2016.csv"), "_2015_")
players_2016 = prefix(loadCSV(csvContext, "2016-2017.csv"), "_2016_")
players_2017 = prefix(loadCSV(csvContext, "2017-2018.csv"), "_2017_")
current_players = prefix(loadCSV(csvContext, "2018-2019.csv"), "current_")

In [57]:
current_2015 = current_players.join(players_2015, current_players.current_G2 == players_2015._2015_G2, how='left')
current_2015_2016 = current_2015.join(players_2016, current_2015.current_G2 == players_2016._2016_G2, how='left')
current_2015_2016_2017 = current_2015_2016.join(players_2017, current_2015_2016.current_G2 == players_2017._2017_G2, how='left')

In [58]:
current_2015_2016_2017.select("current_G2", "current_MV", "_2017_MV", "_2016_MV", "_2015_MV").show()

+--------------------+----------+--------+--------+--------+
|          current_G2|current_MV|_2017_MV|_2016_MV|_2015_MV|
+--------------------+----------+--------+--------+--------+
|         AUDERO-Emil|       6.1|    null|       6|    null|
|     BARDI-Francesco|      5.83|    null|    null|     7.5|
|       BERISHA-Etrit|      6.17|    5.98|    6.25|    5.95|
|     CONSIGLI-Andrea|      6.25|    6.24|    6.28|    6.21|
|      CRAGNO-Alessio|      6.53|     6.2|    null|    null|
|DONNARUMMA-Gianluigi|      6.32|    6.05|    6.42|    6.25|
|DRAGOWSKI-Bartolo...|       6.5|    5.17|       6|    null|
|  FRATTALI-Pierluigi|       6.5|    null|    null|    null|
|   GOLLINI-Pierluigi|      6.18|    6.43|    6.12|    6.21|
|        GOMIS-Alfred|      6.02|    6.34|    null|    null|
|     GUERRIERI-Guido|         5|    null|    null|    null|
|    HANDANOVIC-Samir|       6.3|    6.36|    6.36|    6.39|
|     ICHAZO-Salvador|      6.17|    null|    null|     5.5|
|JANDREI-Chitolina...|  

In [69]:
players_2017.select("_2017_G2").orderBy("_2017_G2").show(2000,truncate=False)

+-----------------------------+
|_2017_G2                     |
+-----------------------------+
|AARONS-Rolando               |
|ABATE-Ignazio                |
|ACERBI-Francesco             |
|ACQUAH-Afriyie               |
|ADJAPONG-Claud               |
|ADNAN-Ali-Kadhim             |
|AJETI-Arlind                 |
|ALBERTO-Luis                 |
|ALBIOL-Tortajada-Raul        |
|ALEX-SANDRO                  |
|ALISSON-Ramses-Becker        |
|ALLAN-Marques-Loureiro       |
|ALVAREZ-Ricardo              |
|ANDERSEN-Joachim             |
|ANDREOLLI-Marco              |
|ANGELLA-Gabriele             |
|ANSALDI-Cristian             |
|ANTEI-Luca                   |
|ANTENUCCI-Mirco              |
|ANTONELLI-Luca               |
|ARMENTEROS-Samuel            |
|ASAMOAH-Kwadwo               |
|ASTORI-Davide                |
|AVENATTI-Felipe              |
|BABACAR-Khouma               |
|BADELJ-Milan                 |
|BAJIC-Riad                   |
|BALIC-Andrija                |
|BANI-Ma